# ConvNN Prime New

In [31]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F

In [32]:

def _calculate_similarity_matrix(self, matrix):
    # p=2 (L2 Norm - Euclidean Distance), dim=1 (across the channels)
    norm_matrix = F.normalize(matrix, p=2, dim=1) 
    similarity_matrix = torch.bmm(norm_matrix.transpose(2, 1), norm_matrix)
    return similarity_matrix


In [ ]:
matrix_x = torch.tensor([[[1, 2, 3, 4], [5, 6, 7, 8]]], dtype=torch.float32)

print("Input Matrix:", matrix_x.shape)
print("Input Matrix:", matrix_x)
print()

a = _calculate_similarity_matrix(None, matrix_x)
print("Similarity Matrix:", a.shape)
print("Similarity Matrix:", a)

# Input Matrix: torch.Size([1, 2, 4])
# Input Matrix: tensor(
#     [
#         [
#             [1., 2., 3., 4.],
#             [5., 6., 7., 8.]
#         ]
#     ]
# )

# Similarity Matrix: torch.Size([1, 4, 4])
# Similarity Matrix: tensor(
#     [
#         [      1.      2.      3.      4.
#          1. [1.0000, 0.9923, 0.9785, 0.9648],
#          2. [0.9923, 1.0000, 0.9965, 0.9899],
#          3. [0.9785, 0.9965, 1.0000, 0.9983],
#          4. [0.9648, 0.9899, 0.9983, 1.0000]
#         ]
#     ]
# )

# Prime Matrix: torch.Size([1, 2, 8])
# Prime Matrix: tensor(
#     [
#         [.     1.      2.      3.      4.
#             [1., 2., 2., 3., 3., 4., 4., 3.],
#             [5., 6., 6., 7., 7., 8., 8., 7.]
#         ]
#     ]
# )

# New Prime Matrix: torch.Size([1, 2, 8])
# New Prime Matrix: tensor(
#     [
#         [.          1.              2.              3.              4.
#             [1.0000, 1.9846, 2.0000, 2.9896, 3.0000, 3.9931, 4.0000, 2.9948],
#             [5.0000, 5.9537, 6.0000, 6.9758, 7.0000, 7.9862, 8.0000, 6.9879]
#         ]
#     ]
# )



Input Matrix: torch.Size([1, 2, 4])
Input Matrix: tensor([[[1., 2., 3., 4.],
         [5., 6., 7., 8.]]])

Similarity Matrix: torch.Size([1, 4, 4])
Similarity Matrix: tensor([[[1.0000, 0.9923, 0.9785, 0.9648],
         [0.9923, 1.0000, 0.9965, 0.9899],
         [0.9785, 0.9965, 1.0000, 0.9983],
         [0.9648, 0.9899, 0.9983, 1.0000]]])


In [36]:

def _prime(self, matrix, magnitude_matrix, K, maximum):
    b, c, t = matrix.shape
    _, topk_indices = torch.topk(magnitude_matrix, k=K, dim=2, largest=maximum)
    topk_indices_exp = topk_indices.unsqueeze(1).expand(b, c, t, K)    
    
    matrix_expanded = matrix.unsqueeze(-1).expand(b, c, t, K).contiguous()
    prime = torch.gather(matrix_expanded, dim=2, index=topk_indices_exp)
    prime = prime.view(b, c, -1)
    return prime

In [37]:
prime = _prime(None, matrix_x, a, K=2, maximum=True)
print("Prime Matrix:", prime.shape)
print("Prime Matrix:", prime)

Prime Matrix: torch.Size([1, 2, 8])
Prime Matrix: tensor([[[1., 2., 2., 3., 3., 4., 4., 3.],
         [5., 6., 6., 7., 7., 8., 8., 7.]]])


In [ ]:

def _prime_new(self, matrix, magnitude_matrix, K, maximum):
    b, c, t = matrix.shape
    topk_values, topk_indices = torch.topk(magnitude_matrix, k=K, dim=2, largest=maximum)
    topk_indices_exp = topk_indices.unsqueeze(1).expand(b, c, t, K)    
    topk_values_exp = topk_values.unsqueeze(1).expand(b, c, t, K)    

    matrix_expanded = matrix.unsqueeze(-1).expand(b, c, t, K).contiguous()
    prime = torch.gather(matrix_expanded, dim=2, index=topk_indices_exp)
    prime = topk_values_exp * prime
    
    prime = prime.view(b, c, -1)
    return prime

In [40]:
new_prime = _prime_new(None, matrix_x, a, K=2, maximum=True)
print("New Prime Matrix:", new_prime.shape)
print("New Prime Matrix:", new_prime)

New Prime Matrix: torch.Size([1, 2, 8])
New Prime Matrix: tensor([[[1.0000, 1.9846, 2.0000, 2.9896, 3.0000, 3.9931, 4.0000, 2.9948],
         [5.0000, 5.9537, 6.0000, 6.9758, 7.0000, 7.9862, 8.0000, 6.9879]]])


### I. New Prime from Farias

In [38]:
def _prime(self, v, qk, K, maximum):
    b, c, t = v.shape 
    topk_values, topk_indices = torch.topk(qk, k=K, dim=-1, largest = maximum)
    topk_indices_exp = topk_indices.unsqueeze(1).expand(b, c, t, K)
    topk_values_exp = topk_values.unsqueeze(1).expand(b, c, t, K)
    v_expanded = v.unsqueeze(-1).expand(b, c, t, K)
    prime = torch.gather(v_expanded, dim=2, index=topk_indices_exp)
    prime = topk_values_exp * prime
    prime = prime.reshape(b, c, -1)
    return prime

# Coordinate Encoding

In [4]:
import torch 
import torch.nn as nn 


In [5]:
coordinate_cache = {}

In [6]:
def _add_coordinate_encoding( x):
    b, c, t = x.shape 
    cache_key = f"{t}_{x.device}"
    if cache_key in coordinate_cache:
        coords_vec = coordinate_cache[cache_key]
    else:
        coords_vec = torch.linspace(start=-1, end=1, steps=t, device=x.device).unsqueeze(0).expand(b, -1)
        coordinate_cache[cache_key] = coords_vec

    expanded_coords = coords_vec.unsqueeze(1).expand(b, -1, -1)
    x_with_coords = torch.cat((x, expanded_coords), dim=1)  
    return x_with_coords
    

In [7]:
x = torch.randn(256, 3, 10)
x_with_coords = _add_coordinate_encoding(x)
print(f"Input shape: {x.shape}")
print(f"With Coordinate Encoding shape: {x_with_coords.shape}")

print(f"Coordinate Cache Size: {len(coordinate_cache)}")
print(f"Coordinate Cache Keys: {list(coordinate_cache.keys())}")

print(f"coordinate cache: {coordinate_cache['10_cpu'][0, ]}")

print(f"x_with_coords: {x_with_coords[0, 3, :]}")  # Print the first channel of the first sample

Input shape: torch.Size([256, 3, 10])
With Coordinate Encoding shape: torch.Size([256, 4, 10])
Coordinate Cache Size: 1
Coordinate Cache Keys: ['10_cpu']
coordinate cache: tensor([-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
         0.7778,  1.0000])
x_with_coords: tensor([-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
         0.7778,  1.0000])


In [11]:
coordinate_cache = {} 
def _add_coordinate_encoding( x):
    b, c, t = x.shape 
    cache_key = f"{b}_{t}_{x.device}"
    if cache_key in coordinate_cache:
        expanded_coords = coordinate_cache[cache_key]
    else:
        coords_vec = torch.linspace(start=-1, end=1, steps=t, device=x.device).unsqueeze(0).expand(b, -1)
        expanded_coords = coords_vec.unsqueeze(1).expand(b, -1, -1)
        coordinate_cache[cache_key] = expanded_coords


    x_with_coords = torch.cat((x, expanded_coords), dim=1)  
    return x_with_coords
    

In [14]:
x = torch.randn(64, 3, 10)
x_with_coords = _add_coordinate_encoding(x)
print(f"Input shape: {x.shape}")
print(f"With Coordinate Encoding shape: {x_with_coords.shape}")

print(f"Coordinate Cache Size: {len(coordinate_cache)}")
print(f"Coordinate Cache Keys: {list(coordinate_cache.keys())}")

print(f"coordinate cache: {coordinate_cache['64_10_cpu'][0, ]}")

print(f"x_with_coords: {x_with_coords[0, 3, :]}")  # Print the first channel of the first sample

Input shape: torch.Size([64, 3, 10])
With Coordinate Encoding shape: torch.Size([64, 4, 10])
Coordinate Cache Size: 2
Coordinate Cache Keys: ['256_10_cpu', '64_10_cpu']
coordinate cache: tensor([[-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
          0.7778,  1.0000]])
x_with_coords: tensor([-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
         0.7778,  1.0000])


## ConvNN 

In [ ]:
"""
ConvNN
Total parameters: 185,150
Trainable parameters: 185,150

Input shape: torch.Size([64, 197, 64])
After Permute: torch.Size([64, 64, 197])
After Split_head: torch.Size([64, 4, 197, 16])
After Batch_Combine: torch.Size([256, 16, 197]) ### [256, 17, 197] added coordinate encoding
After Conv1d: torch.Size([256, 16, 197]) ###    [256, 17, 197] added coordinate encoding
After batch_split: torch.Size([64, 4, 197, 16]) ### [64, 4, 197, 17] added coordinate encoding
After Combine_Heads: torch.Size([64, 64, 197]) ### [64, 68, 197] added coordinate encoding

Output shape: torch.Size([64, 100])
"""

''

## ConvNNAttention

In [ ]:
"""
ConvNNAttention
Total parameters: 83,716
Trainable parameters: 83,716

Input shape: torch.Size([64, 197, 64])
After Split_head: torch.Size([64, 4, 197, 16])
After Batch_Combine: torch.Size([256, 16, 197])
After Conv1d: torch.Size([256, 16, 197])
After permute: torch.Size([256, 197, 16])
After Batch_Split: torch.Size([64, 4, 197, 16])
After Combine_Heads: torch.Size([64, 197, 64])

Output shape: torch.Size([64, 100])

"""

╰─$ python -u "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/vit.py"
Regular Attention
Total parameters: 70,228
Trainable parameters: 70,228
Output shape: torch.Size([64, 100])

ConvNN
Total parameters: 218,546
Trainable parameters: 218,546
Output shape: torch.Size([64, 100])

ConvNNAttention
Total parameters: 71,930
Trainable parameters: 71,930
Output shape: torch.Size([64, 100])

╰─$ python -u "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/vit.py"
Regular Attention
Total parameters: 70,228
Trainable parameters: 70,228
Output shape: torch.Size([64, 100])

ConvNN
Total parameters: 218,295
Trainable parameters: 218,295
Output shape: torch.Size([64, 100])

ConvNNAttention
Total parameters: 71,679
Trainable parameters: 71,679
Output shape: torch.Size([64, 100])